# Topic modelling using BERTopic

## Libraries/data required

In [1]:
# IMPORTS
from bertopic import BERTopic
import pandas as pd
import os
from sklearn.cluster import KMeans, AgglomerativeClustering, OPTICS, SpectralClustering

C:\Users\20210777\Anaconda3\envs\DC3\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\20210777\Anaconda3\envs\DC3\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\20210777\Anaconda3\envs\DC3\lib\site-packages\umap\distances.py:1086: NumbaDepr

In [2]:
# Read the data and perform preprocessing

df = pd.read_csv("data/articles_summary_cleaned.csv", parse_dates=["date"]) # Read data into 'df' dataframe
print(df.shape) # Print dataframe shape

docs = df["summary"].tolist() # Create a list containing all article summaries

# df.head() # Show first 5 dataframe entries
# docs

(18520, 5)


## Fitting BERTopic

This might take a while on a CPU. In the background a pre-trained Large Language Model, called the sentence embedder, is used to convert the articles to a semantic vector space. We then perform clustering in this space.

In [3]:
if os.path.exists('southsudan_model'):
    bertopic = BERTopic.load('southsudan_model')
else:
    bertopic = BERTopic(language="english", calculate_probabilities=True, verbose=True) # Initialize the BERTopic model

    bertopic.fit_transform(docs) # Fit the model to the list of article summaries
    bertopic.save("southsudan_model") # Save the trained model as "southsudan_model"
 
if os.path.exists('kmeans_model'):
    kmeans_model = BERTopic.load('kmeans_model')
else:
    cluster_model = KMeans(n_clusters=10)
    kmeans_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, hdbscan_model=cluster_model) # Initialize the BERTopic model

    kmeans_model.fit_transform(docs) # Fit the model to the list of article summaries
    kmeans_model.save("kmeans_model") # Save the trained model 

if os.path.exists('agglomerative_model'):
    agglomerative_model = BERTopic.load('agglomerative_model')
else:
    cluster_model = AgglomerativeClustering(n_clusters=10)
    agglomerative_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, hdbscan_model=cluster_model) # Initialize the BERTopic model

    agglomerative_model.fit_transform(docs) # Fit the model to the list of article summaries
    agglomerative_model.save("agglomerative_model") # Save the trained model 

if os.path.exists('optics_model'):
    optics_model = BERTopic.load('optics_model')
else:
    cluster_model = OPTICS(min_samples=5)  # Customize the OPTICS parameters as needed
    optics_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, hdbscan_model=cluster_model)

    optics_model.fit_transform(docs)
    optics_model.save("optics_model")

if os.path.exists('spectral_model'):
    spectral_model = BERTopic.load('spectral_model')
else:
    cluster_model = SpectralClustering(n_clusters=10)  # Customize the Spectral Clustering parameters as needed
    spectral_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, hdbscan_model=cluster_model)

    spectral_model.fit_transform(docs)
    spectral_model.save("spectral_model")

In [4]:
#Due to the modularity of the model, there is a lot of randomness that hinders reproducibility of the model.
#To fight this, you can for example set random state in the dimensionality reduction step via the following lines 
#or explore a different approach

#from bertopic import BERTopic
#from umap import UMAP

#umap_model = UMAP(n_neighbors=15, n_components=5, 
#                  min_dist=0.0, metric='cosine', random_state=42)
#topic_model = BERTopic(umap_model=umap_model)

## Interactive visualization of the vector space

As you can see, documents with related topics are close in the space.

In [5]:
# bertopic.visualize_documents(docs) # Create a plot of the topics, this may take a while

### Creating smaller topics

Within our list of topics, we find topics that are semantically closest to 4 keywords:

"Hunger", "Refugees", "Conflict", and "Humanitarian".

**Feel free to change this approach!**

In [6]:
# We create a function to calculate a list of the top n topics related to (a) given keyword(s)

def get_relevant_topics(bertopic_model, keywords, top_n):
    '''
    Retrieve a list of the top n number of relevant topics to the provided (list of) keyword(s)
    
    
    Parameters:
        bertopic_model: a (fitted) BERTopic model object
        
        keywords:   a string containing one or multiple keywords to match against,
                    
                    This can also be a list in the form of ['keyword(s)', keyword(s), ...]
                    
                    In this case a maximum of top_n topics will be found per list element 
                    and subsetted to the top_n most relevant topics.
                    
                    !!!
                    Take care that this method only considers the relevancy per inputted keyword(s) 
                    and not the relevancy to the combined list of keywords.
                    
                    In other words, topics that appear in the output might be significantly related to a 
                    particular element in the list of keywords but not so to any other element, 
                    
                    while topics that do not appear in the output might be significantly related to the 
                    combined list of keywords but not much to any of the keyword(s) in particular.
                    !!!
                    
        top_n: an integer indicating the number of desired relevant topics to be retrieved
        
        
        Return: a list of the top_n (or less) topics most relevant to the (list of) provided keyword(s)
    '''
    
    if type(keywords) is str: keywords = [keywords] # If a single string is provided convert it to list type
    
    relevant_topics = list() # Initilize an empty list of relevant topics
    
    for keyword in keywords: # Iterate through list of keywords
        
        # Find the top n number of topics related to the current keyword(s)
        topics = bertopic_model.find_topics(keyword, top_n = top_n)
        
        # Add the topics to the list of relevant topics in the form of (topic_id, relevancy)
        relevant_topics.extend(
            zip(topics[0], topics[1]) # topics[0] = topic_id, topics[1] = relevancy
        )
    
    
    relevant_topics.sort(key=lambda x: x[1]) # Sort the list of topics on ASCENDING ORDER of relevancy
    
    # Get a list of the set of unique topics (with greates relevancy in case of duplicate topics)
    relevant_topics = list(dict(relevant_topics).items())
    
    
    relevant_topics.sort(key=lambda x: x[1], reverse=True) # Now sort the list of topics on DESCENDING ORDER of relevancy
    
    return relevant_topics[:10] # Return a list of the top_n unique relevant topics

In [7]:
# Get the top 10 topics related to the keywords 'hunger' and 'food insecurity'
relevant_topics = get_relevant_topics(bertopic_model = agglomerative_model, keywords=['hunger', 'food insecurity', 'conflict'], top_n=15)

topic_ids = [el[0] for el in relevant_topics] # Create seperate list of topic IDs

for topic_id, relevancy in relevant_topics: # Print neat list of (topic_id, relevancy) tuples
    print(topic_id, relevancy)
    
df["hunger"] = [t in topic_ids for t in bertopic.topics_] # Add boolean column to df if topic in list of relevant topics

# View the Count, Name, Representation, and Representative Docs for the relevant topics
# bertopic.get_topic_info().set_index('Topic').loc[topic_ids]
# df["hunger"].value_counts()

7 0.38241392
3 0.37269127
1 0.35268196
5 0.33064908
0 0.32147127
2 0.29207075
6 0.26719108
8 0.25609177
4 0.22635308
9 0.20630968


In [8]:
# Get the top 10 topics related to the keywords 'refugees' and 'displaced'
relevant_topics = get_relevant_topics(bertopic_model = agglomerative_model, keywords=['refugees', 'displaced'], top_n=15)

topic_ids = [el[0] for el in relevant_topics] # Create seperate list of topic IDs

for topic_id, relevancy in relevant_topics: # Print neat list of (topic_id, relevancy) tuples
    print(topic_id, relevancy)
    
df["refugees"] = [t in topic_ids for t in bertopic.topics_] # Add boolean column to df if topic in list of relevant topics

# View the Count, Name, Representation, and Representative Docs for the relevant topics
# bertopic.get_topic_info().set_index('Topic').loc[topic_ids]

2 0.5495614
6 0.43483782
5 0.4260249
7 0.41448748
0 0.396513
1 0.38125336
3 0.3779419
8 0.33971408
4 0.33799133
9 0.24752645


In [9]:
# Get the top 10 topics related to the keyword 'humanitarian'
relevant_topics = get_relevant_topics(bertopic_model = agglomerative_model, keywords=['humanitarian'], top_n=15)

topic_ids = [el[0] for el in relevant_topics] # Create seperate list of topic IDs

for topic_id, relevancy in relevant_topics: # Print neat list of (topic_id, relevancy) tuples
    print(topic_id, relevancy)
    
df["humanitarian"] = [t in topic_ids for t in bertopic.topics_] # Add boolean column to df if topic in list of relevant topics

# View the Count, Name, Representation, and Representative Docs for the relevant topics
# bertopic.get_topic_info().set_index('Topic').loc[topic_ids]

2 0.6128026
5 0.55738395
3 0.506997
6 0.4951638
7 0.4824435
0 0.4809367
1 0.47879434
4 0.43003786
8 0.42116326
9 0.3122993


In [10]:
# Get the top 10 topics related to the keywords 'conflict', 'fighting', and 'murder'
relevant_topics = get_relevant_topics(bertopic_model = agglomerative_model, keywords=['conflict', 'fighting', 'murder', 'military'], top_n=15)

topic_ids = [el[0] for el in relevant_topics] # Create seperate list of topic IDs

for topic_id, relevancy in relevant_topics: # Print neat list of (topic_id, relevancy) tuples
    print(topic_id, relevancy)
    
df["conflict"] = [t in topic_ids for t in bertopic.topics_] # Add boolean column to df if topic in list of relevant topics

# View the Count, Name, Representation, and Representative Docs for the relevant topics
# bertopic.get_topic_info().set_index('Topic').loc[topic_ids]

7 0.38241392
3 0.37269127
1 0.35268196
5 0.33064908
0 0.32147127
6 0.26719108
8 0.25948998
2 0.22942087
4 0.22635308
9 0.20630968


In [11]:
# Get the top 10 topics related to the keywords 'politics', 'government', and 'elections'
relevant_topics = get_relevant_topics(bertopic_model = agglomerative_model, keywords=["politics", "government", "elections", "independence"], top_n=15)

topic_ids = [el[0] for el in relevant_topics] # Create seperate list of topic IDs

for topic_id, relevancy in relevant_topics: # Print neat list of (topic_id, relevancy) tuples
    print(topic_id, relevancy)
    
df["politics"] = [t in topic_ids for t in bertopic.topics_] # Add boolean column to df if topic in list of relevant topics

# View the Count, Name, Representation, and Representative Docs for the relevant topics
# bertopic.get_topic_info().set_index('Topic').loc[topic_ids]

0 0.33187532
4 0.3152099
3 0.30114138
8 0.28646624
1 0.2858027
7 0.2836869
6 0.26377165
2 0.26223248
5 0.25914246
9 0.17086889


In [12]:
# Get the top 10 topics related to the keywords 'aid', 'assistance', and 'relief'
relevant_topics = get_relevant_topics(bertopic_model = agglomerative_model, keywords=['aid', 'assistance', 'relief'], top_n=15)

topic_ids = [el[0] for el in relevant_topics] # Create seperate list of topic IDs

for topic_id, relevancy in relevant_topics: # Print neat list of (topic_id, relevancy) tuples
    print(topic_id, relevancy)
    
df["aid"] = [t in topic_ids for t in bertopic.topics_] # Add boolean column to df if topic in list of relevant topics

# View the Count, Name, Representation, and Representative Docs for the relevant topics
# bertopic.get_topic_info().set_index('Topic').loc[topic_ids]

2 0.44024152
6 0.33476654
4 0.3068655
5 0.2967866
1 0.2879024
7 0.2836157
3 0.28057277
0 0.2609885
9 0.21116325
8 0.20132065


In [13]:
original_df = pd.read_csv("data/articles_summary_cleaned.csv", parse_dates=["date"])

# Combine article summaries with the newly created features
df = original_df.merge(
    df[["summary", "hunger", "refugees", "humanitarian", "conflict", "politics", "aid"]],
    how="left",
    left_on="summary",
    right_on="summary",
)

# df.to_csv("data/articles_topics.csv", index=False) # Save DataFrame to articles_topics.csv

In [14]:
print(len(df))
print(len(df[(df["hunger"]==False) & (df["refugees"] == False) & (df["humanitarian"] == False) & (df["conflict"] == False)]))
print(len(df[(df["hunger"]==False) & (df["refugees"] == False) & (df["humanitarian"] == False) & (df["conflict"] == False) & (df["politics"] == False) & (df["aid"] == False)]))

18520
16224
16224


In [15]:
unsorted = df[(df["hunger"]==False) & (df["refugees"] == False) & (df["humanitarian"] == False) & (df["conflict"] == False) & (df["politics"] == False)]
unsorted.head()

,summary,date,location_article,lat,lng,hunger,refugees,humanitarian,conflict,politics,aid
0,The article discusses the passing of the new C...,2011-07-07,Juba,4.859363,31.571250,False,False,False,False,False,False
1,The article discusses the military actions tak...,2011-07-03,Abyei,9.838551,28.486396,False,False,False,False,False,False
3,The article discusses the upcoming independenc...,2011-07-04,South Sudan,6.876992,31.306979,False,False,False,False,False,False
4,The article discusses the need for South Sudan...,2011-07-02,Juba,4.859363,31.571250,False,False,False,False,False,False
5,The article discusses the United States' respo...,2011-07-06,Addis Ababa,8.980603,38.757761,False,False,False,False,False,False


In [16]:
# if os.path.exists('southsudan_model_politics'):
#     bertopic = BERTopic.load('southsudan_model_politics')
# else:
#     bertopic = BERTopic(language="english", calculate_probabilities=True, verbose=True) # Initialize the BERTopic model
# 
#     bertopic.fit_transform(unsorted["summary"].tolist()) # Fit the model to the list of article summaries
#     bertopic.save("southsudan_model_politics") # Save the trained model as "southsudan_model"

In [17]:
# uns_list = unsorted["summary"].tolist()
# #Write the list of uns_list to text file using utf-8 encoding
# with open("data/uns_list.txt", "w", encoding="utf-8") as f:
#     for s in uns_list:
#         f.write(str(s) +"\n")

In [ ]:
#Refit models on unsorted data
if os.path.exists('refit_kmeans'):
    refit_kmeans = BERTopic.load('refit_kmeans')
else:
    refit_kmeans = BERTopic(language="english", calculate_probabilities=True, verbose=True, hdbscan_model=KMeans(n_clusters=10))
    refit_kmeans.fit_transform(unsorted["summary"].tolist())
    refit_kmeans.save("refit_kmeans")

if os.path.exists('refit_agglomerative'):
    refit_agglomerative = BERTopic.load('refit_optics')
else:
    refit_agglomerative = BERTopic(language="english", calculate_probabilities=True, verbose=True, hdbscan_model=AgglomerativeClustering(n_clusters=10))
    refit_agglomerative.fit_transform(unsorted["summary"].tolist())
    refit_agglomerative.save("refit_agglomerative")

if os.path.exists('refit_spectral'):
    refit_spectral = BERTopic.load('refit_spectral')
else:
    refit_spectral = BERTopic(language="english", calculate_probabilities=True, verbose=True, hdbscan_model=SpectralClustering(n_clusters=10))
    refit_spectral.fit_transform(unsorted["summary"].tolist())
    refit_spectral.save("refit_spectral")


Batches:   0%|          | 0/507 [00:00<?, ?it/s]

2023-09-25 13:21:24,581 - BERTopic - Transformed documents to Embeddings
2023-09-25 13:21:50,029 - BERTopic - Reduced dimensionality
  File "C:\Users\20210777\Anaconda3\envs\DC3\lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(
2023-09-25 13:21:50,482 - BERTopic - Clustered reduced embeddings


Batches:   0%|          | 0/507 [00:00<?, ?it/s]

In [ ]:
current_model = refit_agglomerative

In [ ]:
# Define a list of keyword sets, each with keywords and a label
keyword_sets = [
    (['hunger', 'food insecurity', 'conflict'], 'hunger'),
    (['refugees', 'displaced'], 'refugees'),
    (['humanitarian'], 'humanitarian'),
    (['conflict', 'fighting', 'murder', 'military'], 'conflict'),
    (["politics", "government", "elections", "independence"], 'politics')
]

for keywords, label in keyword_sets:
    # Get the top 10 topics related to the current set of keywords
    relevant_topics = get_relevant_topics(bertopic_model=current_model, keywords=keywords, top_n=15)
    
    # Create a list of topic IDs
    topic_ids = [el[0] for el in relevant_topics]
    
    # Print the relevant topics
    print(f"Top 10 topics related to '{label}':")
    for topic_id, relevancy in relevant_topics:
        print(topic_id, relevancy)
    
    # Add a boolean column to 'unsorted' DataFrame if the topic is in the list of relevant topics
    unsorted[label] = [t in topic_ids for t in bertopic.topics_]